# Julia 型別 (Types) 系統簡介

## 1. 型別 (Type) 的宣告

型別的宣告是使用 `::` 運算子  可以針對變數、表達式 (expression)、與函式進行型別的宣告.

### 變數 (variable)

宣告變數型別, 只能針對 local 變數, 若宣告 global 變數的型別時會產生錯誤.

In [1]:
local x::Int8 = 10

10

In [2]:
# global 變數
y::Int64 = 10

ErrorException: syntax: type declarations on global variables are not yet supported

In [3]:
function foo()
    x::Int32 = 100
    x
end

foo()

100

呼叫 `foo()` 後, 我們可以看到回傳值的型別是 `Int32`

In [4]:
# ans 是 Julia 語言保留字, 代表的是目前最後計算值
typeof(ans)

Int32

### 表達式 (expression)

#### Type assertion
- The `::` operator can be used to attach type annotations to expressions and variables in programs. There are two primary reasons to do this:

    1. As an assertion to help confirm that your program works the way you expect,
    2. To provide extra type information to the compiler, which can then improve performance in some cases

- the `::` operator is read as **"is an instance of"**
- to assert that the value of the expression on the left is an instance of the type on the right 
- When the type on the right is 
    - concrete, the value on the left must have that type as its implementation – recall that all concrete types are final, so no implementation is a subtype of any other. 
    - abstract, it suffices for the value to be implemented by a concrete type that is a subtype of the abstract type. 

If the type assertion is not true, an exception is thrown, otherwise, the left-hand value is returned

In [2]:
(1+2)::Float64

TypeError: TypeError: in typeassert, expected Float64, got Int64

In [4]:
(1+2)::Int64

3

As part of a **local declaration**, it declares the variable to always have the specified type

In [7]:
# here a is global
a::Int8 = 12

ErrorException: syntax: type declarations on global variables are not yet supported

In [8]:
function foo()
    # here a is local
    a::Int8 = 12
end
foo()

12

In [26]:
local Lc::Int8 = 12

12

In [11]:
local Lc2
Lc2::Int8 = 11

ErrorException: syntax: type declarations on global variables are not yet supported

### 函式 (function)

若宣告函式回傳型別的話, **回傳**值會進行**轉型為指定的型別**, 若無法轉型的話則會產生錯誤.

Returning from this function behaves just like an assignment to a variable with a declared type: the value is always converted to Float64.

In [12]:
function foo()::Float64
    return 2 / 2
end

foo()

1.0

## 2. 常用函式與型別階層

下圖為 `Number` 及其子型別，`Any` 為所有型別的父型別。

![](Julia_Number.png)

呼叫 `supertype()` 函式查看某一型別的父型別.

In [6]:
supertype(Real)

Number

呼叫 `subtypes()` 函式查看某一型別的子型別.

In [7]:
subtypes(Number)

2-element Array{Any,1}:
 Complex
 Real

型別的父、子關係，是使用 `<:` 與 `>:` 運算子來指定。

查看型別之間是否為父子關係。

In [10]:
<:(Integer, Number)

true

In [11]:
# 也可以直接使用運算子
Integer <: AbstractFloat

false

In [10]:
Number >: Real >: AbstractFloat >: Float64

true

In [11]:
# Integer 也是 Int64 的父型別
Integer >: Int64

true

## 3. 抽象型別 (Abstract Type)

抽象型別有幾個特性：
- 無法被實例化 (Instantiate)
    - (in c#) The process of creating an object from a class is called instantiation because an object is an instance of a class.
- 抽象是可以延伸的, 可以有多層次
- 可以運用抽象型別來撰寫泛用 (generic) 函式，以做為函式的基本行為 (behavior)

抽象型別的最上層是 `Any`, 也是所有型別的最上層父型別; 抽象型別的最下層則是 `Union{}`, 也是所有型別的子型別.

以 `Number` 型別為例, 可以看到其為所有數值型別的父型別, 而且是抽象型別.

In [13]:
? Number

search: Number LineNumberNode VersionNumber



```
Number
```

Abstract supertype for all number types.


### 抽象型別的宣告

`abstract type <<name>> end`

宣告時可使用 `<:` 運算子指定其父型別. 如果**沒有指定的話, 則父型別為 `Any`**.

`abstract type <<name>> <: <<supertype>> end`

In [14]:
abstract type YetAnotherNumber <: Number end

下列範例是當函式沒有指定型別時其型別為抽象型別 `Any`, 傳入整數引數 (argument) 值時, Julia 會內部定義並編譯符合整數引數的 Method 並執行, 所以仍然可以正確地計算出整數之回傳值.

In [15]:
function xyplus(x, y)
    x + y
end

xyplus (generic function with 1 method)

In [16]:
xyplus(2, 5)

7

## 4. 原始型別 (Primitive Type)

### 原始型別的宣告

`primitive type <<name>> <<bits>> end`

宣告時可使用 `<:` 運算子指定其父型別. 如果沒有指定的話, 則父型別為 `Any`.

`primitive type <<name>> <: <<supertype>> <<bits>> end`

bits 數字是指定型別需要的儲存空間.

範例: Int64 的宣告

`primitive type Int64 <: Signed 64 end`

In [16]:
primitive type AnotherNumericType <: Signed 64 end

## 5. 複合型別 (Composite Type)

複合型別是用關鍵字 `struct` 來進行宣告，成員可以指定型別或不指定，不指定即為 `Any` 型別；成員的型別也可以是另一個複合型別。

`struct` 可分為可變 (mutable) 與不可變的，其差異在於在實例化 (instantiate) 之後 mutable struct 的成員值是可以被改變的，而不可變複合型別成員值不可以被改變，嘗試改變時會產生錯誤。

下例為宣告宣告不可變的複合型別。

In [13]:
struct Bar
    m::Int64
    n::Float64
end

In [14]:
struct Foo
    aa::Bar
    bb::Int
    cc::Float64
end

In [15]:
b1 = Foo(Bar(3,2),1,1.5)
b1.aa.m

3

The behavior of type assertion in struct seems to be just like `local Lc::Int8 = 12.0` and `local Lc::Int8 = 12.1`

In [27]:
# automatically convert the input argument to the asserted type
b2 = Foo(Bar(3.0,0),1,1.5)

Foo(Bar(3, 0.0), 1, 1.5)

In [28]:
# error occurred if it cannot be converted to the asserted type
b2 = Foo(Bar(3.1,0),1,1.5)

InexactError: InexactError: Int64(3.1)

複合型別的成員是可以被存取的，例如：

In [88]:
b1.cc

1.5

In [6]:
b1.aa.m = 5 # immutable in default

ErrorException: setfield! immutable struct of type Bar cannot be changed

可變的複合型別，在宣告時使用 `mutable` 保留字。

In [7]:
mutable struct Foo2
    aa::Bar
    bb::Int
    cc::Float64
end

ErrorException: syntax: "aa::Bar = 1" inside type definition is reserved

In [28]:
b2 = Foo2(Bar(1, 2.0), 3, 4.0)

Foo2(Bar(1, 2.0), 3, 4.0)

In [29]:
b2.bb = 2

2

In [30]:
b2.aa

Bar(1, 2.0)

在複合型別內的複合型別成員透過下列語法存取。

In [31]:
b2.aa.n

2.0

呼叫 `dump()` 函式可以查看複合型別的內部結構。

In [19]:
dump(Foo)

Foo <: Any
  aa::Bar
  bb::Int64
  cc::Float64


In [20]:
dump(Bar)

Bar <: Any
  m::Int64
  n::Float64


在複合型別內的複合型別成員透過下列語法存取。

In [21]:
fieldnames(Foo)

(:aa, :bb, :cc)

## 6. 型別聯合 (Union)

型別聯合是透過 Union 的方式，可以讓物件的型別限定在數個指定的型別之內，但又不需要使用更高層級或是抽象型別來宣告，例如：下列語法示範了，如果我們希望型別可能是 Int64、Int32、或是 Float64 時，可以透過 Union 來達到。

In [37]:
Int64OrInt32OrFloat64 = Union{Int64, Float64, Int32}

Union{Float64, Int32, Int64}

In [38]:
Int64OrInt32OrFloat64(1)

1

In [41]:
not_in_union = 5//3;
println(typeof(not_in_union))
Int64OrInt32OrFloat64(not_in_union)

Rational{Int64}


MethodError: MethodError: no method matching Union{Float64, Int32, Int64}(::Rational{Int64})
Closest candidates are:
  Union{Float64, Int32, Int64}(::T) where T<:Number at boot.jl:715
  Union{Float64, Int32, Int64}(!Matched::Complex) where T<:Real at complex.jl:37
  Union{Float64, Int32, Int64}(!Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:243
  ...

如果型別不在 Union 中，則會出現 `TypeError`。

In [42]:
a = Int8(1)
a::Int64OrInt32OrFloat64

TypeError: TypeError: in typeassert, expected Union{Float64, Int32, Int64}, got Int8

In [123]:
function a_function()
    a2::Int64OrInt32OrFloat64 = 52 # or try 52.0
    typeof(a2)
end
a_function()

Int64

In [53]:
typeof(Int64OrInt32OrFloat64)

Union

可以看到 Union 的型別是 `DataType`。

In [54]:
typeof(Union)

DataType

其父型別為 `Type{T}`，是參數化型別。

In [55]:
supertype(Union)

Type{T}

In [83]:
subtype(Union)

UndefVarError: UndefVarError: subtype not defined

In [85]:
Any >: Type >: Union 

true

In [105]:
println("typeof(Any) is $(typeof(Any))")
println("typeof(Type) is $(typeof(Type))")
println("typeof(Union) is $(typeof(Union))")

typeof(Any) is DataType
typeof(Type) is UnionAll
typeof(Union) is DataType


如果 `Union` 內參數為空，即 `Union{}`，則其為特殊型別，是最底層的型別，是所有型別的子型別。

In [76]:
typeof(Union{})

Core.TypeofBottom

In [77]:
# in comparison
typeof(Union{Int64})

DataType

In [78]:
# in comparison
typeof(Union{Int64,Int32})

Union

In [79]:
subtypes(Union{})

0-element Array{Type,1}

In [80]:
# in comparison
subtypes(Union{Int64,Int32})

0-element Array{Type,1}

## 7. 型別參數化 (Parametric Types)

- 型別參數化可以彈性地型別，而不影響程式的彈性及重覆使用性，也能降低程式的複雜度。型別參數化分為兩個部分介紹：
    - 複合型別參數化
    - 抽象型別參數化
- 型別參數化相較於抽象型別可以使程式碼被執行得更快(因為，以下`struct Point`為例，`x`的型別在實例化的時候就會被確定，之後投入運算會更有效率)，但代價是實例化後就不能變更域值的型別(見julia 1.4 doc 43.5 Type Declarations)

### 複合型別參數化

複合型別參數化的宣告語法：`struct <<name>>{T}`

In [130]:
struct Point{T}
    x::T
end

In [1]:
typeof(5//2)

Rational{Int64}

In [131]:
x1 = 5//2
t1 = Point(x1)

Point{Rational{Int64}}(5//2)

In [133]:
t1 * 2

MethodError: MethodError: no method matching *(::Point{Rational{Int64}}, ::Int64)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  *(!Matched::Complex{Bool}, ::Real) at complex.jl:309
  *(!Matched::Missing, ::Number) at missing.jl:115
  ...

In [134]:
x1 = 1e10
t1 = Point(x1)

Point{Float64}(1.0e10)

In [127]:
typeof(t1)

Point{Float64}

In [128]:
typeof(Point)

UnionAll

`Vector` to create an array is in fact a struct (parametric types)

In [16]:
dump(Vector)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Core.TypeofBottom Union{}
    ub: Any
  body: Array{T,1} <: DenseArray{T,1}


In [17]:
Vector.var.lb

Union{}

In [18]:
Vector.var.name

:T

In [21]:
Vector.body

Array{T,1}

In [22]:
Vector{Int64}

Array{Int64,1}

In [23]:
# T is not a type such as Int64 or Any
# it is also OK to use other words/letters to replace T
struct PointAnyType{Int64}
    x::T
end

UndefVarError: UndefVarError: T not defined

In [28]:
struct PointF64
    x::Float64
end
typeof(PointF64) # compare to typeof(Point) above

DataType

In [110]:
Point64

Point64

In [111]:
Any

Any

In [112]:
Int64

Int64

### 抽象型別參數化

下例是抽象型別 `Pointy` 參數化，而 `Pointy` 有三個子型別，分別代表一維、二維、三維座標型別而有不同的成員。型別階層如下圖：

![](pointy.png)

抽象化型別參數化的宣告語法：`abstract type <<name>>:{T} end`

In [29]:
abstract type Pointy{T} end

In [30]:
Pointy

Pointy

In [31]:
struct Point1D{T} <: Pointy{T}
    x::T
end

In [32]:
struct Point2D{T} <: Pointy{T}
    x::T
    y::T
end

In [54]:
struct Point3D{T} <: Pointy{T}
    x::T
    y::T
    z::T
    struct N3DS
        switch::Int
    end    ## struct in struct won't cause error but is effectiveless at all.
end

In [55]:
a = Point3D{Int64}(3,2,1)

Point3D{Int64}(3, 2, 1)

In [56]:
typeof(a)

Point3D{Int64}

In [57]:
dump(Point3D)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Core.TypeofBottom Union{}
    ub: Any
  body: Point3D{T} <: Pointy{T}
    x::T
    y::T
    z::T


Comparing to `Vector`

In [58]:
dump(Vector)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Core.TypeofBottom Union{}
    ub: Any
  body: Array{T,1} <: DenseArray{T,1}


In [59]:
b = Vector{Int64}(1:3)

3-element Array{Int64,1}:
 1
 2
 3

In [39]:
typeof(b)

Array{Int64,1}

函式之引數宣告為抽象型別，在函式內判斷傳入的複合型別為何，並印出傳入的成員值。

透過 `isa()` 函式判斷型別，若型別相同則為 `true`，若不同則為 `false`。

In [159]:
function print_members(p::Pointy)
    if isa(p, Point1D)
        println(p.x)
    elseif isa(p, Point2D)
        println(p.x, " ", p.y)
    else
        println(p.x, " ", p.y, " ", p.z)
    end
end

print_members (generic function with 1 method)

In [160]:
p2 = Point2D(1.0, 2.0)

Point2D{Float64}(1.0, 2.0)

In [161]:
print_members(p2)

1.0 2.0


Also see:
https://medium.com/@dboyliao/julia-%E7%8E%A9%E6%A8%82%E7%AD%86%E8%A8%98-type-system-378edeab7577

# julia 玩樂筆記: Type System
## TL;DR

- 分別有 abstract type 與 concrete type 兩種
    - abstract type 可以被繼承，但不能生成實例；
    - concrete type 可以生成實例，但不能被繼承。
    - Julia 主要是透過 abstract type 去確立各種 type 之間的階層關係，而 concrete type 才真的實作了物件如何生成。
- 不可以多重繼承。
    - 每個 type 的 supertype 都只會有一個
- UnionAll: 所有帶有未知參數的 parametric type 的 type
- Type: 一個 singleton parametric type，isa(A, Type{B}) = true if and only if A === B
- DataType: 所有未帶有任何未知參數的 type 的 type
- Any: 所有 type 的 supertype
- Union{}: 所有 type 的 subtype，不會有任何 type 會是 Union{} 的 subtype 。

> **(!) 為了幫助理解，本資源可能含有不正確的資訊。**
> **Please also refer to https://yuehhua.github.io/2017/06/05/state-centered-behavior-centered-oop/?fbclid=IwAR1knUVQubIC_MAZt1hcOJv1Z6w42VnQgfVer_yz7dEmogYnTBwQEEcCkVA for a more canonical interpretation.**

In [162]:
function supertypes(type)
    if type === Any
        return [Any]
    end
    supers = []
    while true
        if type === Any
            break
        end
        type = supertype(type)
        push!(supers, type)
    end
    supers
end

supertypes (generic function with 1 method)

In [163]:
supertypes(Int64)

5-element Array{Any,1}:
 Signed
 Integer
 Real
 Number
 Any

In [164]:
# that is
Int64 <: Signed <: Integer <: Real <: Number <: Any

true

In [166]:
# or use the built-in function:
Base.show_supertypes(Int64)

Int64 <: Signed <: Integer <: Real <: Number <: Any

- Any 是所有 type 的 supertype，是 type graph 最頂端
- Union{} 是 type graph 的最底層。也就是說沒有任何物件的型別會是 Union{} ，而 Union{} 會是所有 type 的 subtype

In [197]:
subtypes(Union)

0-element Array{Type,1}

In [199]:
subtypes(Union{Int64,Float64})

0-element Array{Type,1}

## Type, DataType and Union

在 Julia 裡所有的值都是 object ，所有的 object 都會有個 type 。 
- 1.0 (值)是一個 Float64 (type) 
- 宣告函數時可以指定變數的型別，這能幫助 Julia compiler 優化效能， 譬如說 foo(x::Float64) 、 foo(x::Int32) 
- `Int64`, `Float32`... belong to **concrete type**, are instances of `DataType`
    - they (e.g `Int64`) have a certain supertype,
    - they can create instances, such as x = Int64; y = Int64;

in julia, every value/parameter is an object, and every object has a type
- instances can be created with *parameters and values*. E.g. Complex{Int64}.
- a *parameter or value* is an object (an instance created, and can create instance).

In [204]:
x = 3 # x is an instance of Int64 with its value 3
y = 4 # x is another instance of Int64 with its value 4
typeof(y),typeof(x)

(Int64, Int64)

In [205]:
x = Int64 # an instance of DataType
typeof(x)

DataType

In [206]:
typeof(Int64),typeof(Any),typeof(Union) # these 'types' are instances of Class 'DataType'

(DataType, DataType, DataType)

In [207]:
Base.show_supertypes(DataType)

DataType <: Type{T} <: Any

In [208]:
subtypes(DataType)

0-element Array{Type,1}

field name of DataType and Int64
- if **`xxx.mutable`** is true, its **field value can be modified**.

In [213]:
Int64.name, Int64.size, Int64.super, Int64.mutable

(Int64, 8, Signed, false)

In [217]:
DataType.name, DataType.size, DataType.super, DataType.mutable 
# Although DataType is mutable, but don't change its field values, it may crashed.

(DataType, 96, Type{T}, true)

 `Type{T}`
 - parametric singleton type
 - 在給定一個 T (例如 Float64) ，它唯一的實例就會是 T 
 - is abstract
     - `Float64 === Type{Float64}` is true, because Type{Float64} can never has an instance.

In [236]:
isa(Float64,Type{Float64})

true

UnionAll 即所有帶有未知參數的參數化型別的 type 。但當一個 parametric type 中的所有參數都為已知時，即變成一個 DataType 的實例；反之，若有任何未知參數，仍舊會是 UnionAll 的實例。

In [244]:
typeof(Type)

UnionAll

In [245]:
typeof(Type{Float64})

DataType

In [246]:
subtypes(Type{Float64})

1-element Array{Any,1}:
 Type{Float64}

In [247]:
Base.show_supertypes(Type{Float64})

Type{Float64} <: Any

In [248]:
subtypes(Type)

4-element Array{Any,1}:
 Core.TypeofBottom
 DataType
 Union
 UnionAll

In [249]:
Base.show_supertypes(Type)

MethodError: MethodError: no method matching show_supertypes(::Type{Type})
Closest candidates are:
  show_supertypes(!Matched::IO, !Matched::DataType) at show.jl:595
  show_supertypes(!Matched::DataType) at show.jl:602

因為 Julia 是採 multiple dispatching ，所以當在設計 package 並且需要給 type annotation 時，以上四種宣告方式會被呼叫的狀況與方式都不盡相同。

- foo(x :: Float64)
    - 適用於 x 是 Float64 的實例時，例如 x = 1.0
- foo(x :: DataType)
- foo(x :: Type)
- foo(x :: UnionAll)
    - 適用的狀況是 x 是 UnionAll 的實例時，例如 x = Complex